# C-Town year simulation using weeks

This notebook can be used to run a C-Town simulation for 9 months in a "week-by-week" (weeks of 10-days) basis
The notebook reads from .csv files to define the tank level initial conditions for the week and the demand patterns for that week. 
The tank initial levels are stored in "tank_initial_conditions" and the demand points are defined in "starting_demand_points" and "three_years_demands_ctown"

In [1]:
# load module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import wntr
import wntr_utils
from sklearn.preprocessing import MinMaxScaler
from random import shuffle
import csv
import random

In [2]:
week_index = 1
total_demands = pd.read_csv('three_year_demands_ctown.csv', index_col=0)
demand_starting_points = pd.read_csv('starting_demand_points.csv', index_col=0)
initial_tank_levels = pd.read_csv('tank_initial_conditions.csv', index_col=0)
network_delays = pd.read_csv('network_links_delay.csv', index_col=0)
network_losses = pd.read_csv('network_loss_small.csv', index_col=0)

In [3]:
# load EPANET model
inp_file = 'ctown_map_with_controls.inp'
wn = wntr.network.WaterNetworkModel(inp_file)

C:\ProgramData\Anaconda3\lib\site-packages\wntr\epanet\io.py:2085: UserWarning:

Not all curves were used in "ctown_map_with_controls.inp"; added with type None, units conversion left to user



In [5]:
wn.options.time.duration=300

In [9]:
wn.options.time.duration

300

In [7]:
wn.options.time.hydraulic_timestep

300

In [4]:
week_index = 1
week_start = demand_starting_points.iloc[week_index][0]

In [5]:
week_demands = total_demands.loc[week_start:week_start+239:]

In [6]:
# Replace the demand patterns with the random patterns for this week
names, patterns =[],[]
for name, pat in wn.patterns():  
    pat.multipliers = week_demands[name].values.tolist()

In [27]:
wn.options.time.duration

864000

In [28]:
for pat in wn.patterns():  
    print(pat)

('DMA1_pat', <Pattern 'DMA1_pat', multipliers=array([0.547, 0.58 , 0.557, 0.513, 0.497, 0.524, 0.398, 0.515, 0.429, 0.337, 0.347, 0.354, 0.389, 0.489, 0.696, 0.724, 0.724, 0.676, 0.643, 0.579, 0.57 , 0.533, 0.565, 0.543, 0.555, 0.549, 0.547, 0.5  , 0.611, 0.523, 0.409, 0.482, 0.44 , 0.34 , 0.33 , 0.364, 0.361, 0.498, 0.69 , 0.716, 0.626,
       0.628, 0.576, 0.605, 0.571, 0.526, 0.5  , 0.483, 0.495, 0.611, 0.64 , 0.59 , 0.512, 0.472, 0.416, 0.497, 0.463, 0.31 , 0.339, 0.344, 0.448, 0.563, 0.641, 0.71 , 0.692, 0.655, 0.574, 0.534, 0.495, 0.534, 0.575, 0.603, 0.644, 0.535, 0.573, 0.531, 0.491, 0.424, 0.392, 0.486, 0.483, 0.35 ,
       0.327, 0.38 , 0.398, 0.534, 0.635, 0.799, 0.668, 0.716, 0.632, 0.586, 0.534, 0.547, 0.558, 0.518, 0.611, 0.628, 0.588, 0.542, 0.536, 0.498, 0.429, 0.503, 0.444, 0.358, 0.337, 0.341, 0.398, 0.545, 0.611, 0.751, 0.791, 0.69 , 0.647, 0.53 , 0.518, 0.495, 0.525, 0.517, 0.56 , 0.563, 0.6  ,
       0.556, 0.553, 0.469, 0.451, 0.65 , 0.461, 0.378, 0.369, 0.342, 0.

In [29]:
# Replace the initial tank levels with the values stored in the .csv file
for i in range(1,8):    
    wn.get_node('T' + str(i)).init_level = float(initial_tank_levels.iloc[week_index]['T'+ str(i)])

In [9]:
initial_tank_levels.iloc[week_index]

T1    5.319035
T2    1.294841
T3    4.277594
T4    3.547004
T5    3.844998
T6    4.195264
T7    1.271854
Name: 1, dtype: float64

In [32]:
wn.write_inpfile('ctown_year_simulation_week_1.inp')

In [7]:
# Simulate hydraulics
sim = wntr.sim.WNTRSimulator(wn,mode='PDD')

In [8]:
# this would run the simulation. 
# Mohammad, using this notebook as base you could change the notebook to run all the 38 weeks. Notice that the variable week_index 
# is the one controlling which week is being simulated
results_new = sim.run_sim()

In [15]:
pressure_results = results_new.node['pressure']
pressure_results.to_csv('test_ctown_week_1_wntr.csv')